# APIM ❤️ OpenAI

## アクセス制御ラボ
![flow](../../images/access-controlling.gif)

特定のユーザーやクライアントによるOpenAPI APIへのより細かいアクセスを可能にするために、アイデンティティプロバイダーを使用して[OAuth 2.0認証機能](https://learn.microsoft.com/en-us/azure/api-management/api-management-authenticate-authorize-azure-openai#oauth-20-authorization-using-identity-provider)を試すためのプレイグラウンド。

### 目次
- [0️⃣ ノートブック変数の初期化](#0)
- [1️⃣ Microsoft Entra IDでのアプリ登録の作成](#1)
- [2️⃣ Azureリソースグループの作成](#2)
- [3️⃣ 🦾 Bicepを使用したデプロイの作成](#3)
- [4️⃣ デプロイの出力を取得](#4)
- [5️⃣ アクセストークンを取得するためのデバイスフローの作成](#5)
- [6️⃣ トークンを取得してグラフAPIをクエリ](#6)
- [🧪 直接HTTPコールを使用したAPIのテスト](#requests)
- [🧪 Azure OpenAI Python SDKを使用したAPIのテスト](#sdk)
- [🗑️ リソースのクリーンアップ](#clean)

### 前提条件
- [Python 3.8以降のバージョン](https://www.python.org/)がインストールされていること
- [Pandasライブラリ](https://pandas.pydata.org/)がインストールされていること
- [VS Code](https://code.visualstudio.com/)がインストールされ、[Jupyterノートブック拡張機能](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)が有効になっていること
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli)がインストールされていること
- Contributor権限を持つ[Azureサブスクリプション](https://azure.microsoft.com/en-us/free/)があること
- [Azure OpenAIへのアクセス](https://aka.ms/oai/access)が許可されていること、またはモックサービスを有効にすること
- [Azure CLIでAzureにサインイン](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)していること

<a id='0'></a>
### 0️⃣ ノートブック変数の初期化

- リソースはサブスクリプションIDに基づいた一意の文字列でサフィックスされます
- ロケーションパラメータは、[Azureリージョンごとの製品の利用可能性](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)に基づいて調整してください
- OpenAIモデルとバージョンは、[リージョンごとの利用可能性](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)に基づいて調整してください

In [2]:
! az login --tenant "fdb5bdfe-92ed-4d5a-b697-a66439451bda"

A web browser has been opened at https://login.microsoftonline.com/fdb5bdfe-92ed-4d5a-b697-a66439451bda/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.

Retrieving subscriptions for the selection...

[Tenant and subscription selection]

No     Subscription name    Subscription ID                       Tenant
-----  -------------------  ------------------------------------  ------------------------------------
[1] *  koheisaito           59e7c4a1-e556-4873-87e6-ba29af832e01  fdb5bdfe-92ed-4d5a-b697-a66439451bda

The default is marked with an *; the default tenant is 'fdb5bdfe-92ed-4d5a-b697-a66439451bda' and subscription is 'koheisaito' (59e7c4a1-e556-4873-87e6-ba29af832e01).

Select a subscription and tenant (Type a number or Enter for no changes): ^C


In [1]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
apim_resource_sku = "Basicv2"
openai_resources = [ {"name": "openai1", "location": "swedencentral"}, {"name": "openai2", "location": "francecentral"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_resources_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ {"name": "openaimock1", "endpoint": "https://openaimock1.azurewebsites.net"}, {"name": "openaimock2", "endpoint": "https://openaimock2.azurewebsites.net"} ]

log_analytics_name = "workspace"
app_insights_name = 'insights'

app_registration_name = "ai-gateway-openai-app"


<a id='1'></a>
### 1️⃣ Microsoft Entra IDでのアプリ登録の作成
以下のコマンドはクライアントアプリケーション登録を作成します

In [3]:
cmd_stdout = ! az account show --query homeTenantId --output tsv
tenant_id = cmd_stdout.n

cmd_stdout = ! az ad app create --display-name {app_registration_name} --query appId --is-fallback-public-client true --output tsv
client_id = cmd_stdout.n


<a id='2'></a>
### 2️⃣ Azureリソースグループの作成
このラボでデプロイされるすべてのリソースは、指定されたリソースグループに作成されます。既存のリソースグループを使用する場合は、このステップをスキップしてください。

In [4]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

✅ Azure Resource Group  lab-access-controlling  created ⌚  11:34:31.567287


<a id='3'></a>
### 3️⃣ 🦾 Bicepを使用したデプロイの作成

このラボでは、[Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep)を使用して、デプロイされるすべてのリソースを宣言的に定義します。異なる構成を試すために、パラメータや[main.bicep](main.bicep)を直接変更してください。

In [5]:
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")
elif len(mock_webapps) > 0:
    backend_id = mock_backend_pool if len(mock_backend_pool) > 1 else mock_webapps[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id).replace("{aad-client-application-id}", client_id).replace("{aad-tenant-id}", tenant_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name }
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


A new Bicep release is available: v0.30.23. Upgrade now by running "az bicep upgrade".
/Users/koheisaito/work/Microsoft/repos/AI-Gateway/labs/access-controlling/main.bicep(282,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is an inaccuracy in the documentation, please report it to the Bicep Team. [https://aka.ms/bicep-type-issues]
/Users/koheisaito/work/Microsoft/repos/AI-Gateway/labs/access-controlling/main.bicep(299,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is an inaccuracy in the documentation, please report it to the Bicep Team. [https://aka.ms/bicep-type-issues]

 | Finished .. - Starting ..{
  "id": "/subscriptions/59e7c4a1-e556-4873-87e6-ba29af832e01/resourceGroups/lab-access-controlling/providers/Microsoft.Resources/deployments/access-controlling",
  "location": null,
  "name": "access-controlling",
  "propertie

867

<a id='4'></a>
### 4️⃣ デプロイの出力を取得

テストの準備が整う前に、ゲートウェイURLとサブスクリプションを取得する必要があります。

In [6]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimServiceId.value -o tsv
apim_service_id = deployment_stdout.n
print("👉🏻 APIM Service Id: ", apim_service_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)

👉🏻 APIM Service Id:  /subscriptions/59e7c4a1-e556-4873-87e6-ba29af832e01/resourceGroups/lab-access-controlling/providers/Microsoft.ApiManagement/service/apim-fflvbx3bpkbqw
👉🏻 API Gateway URL:  https://apim-fflvbx3bpkbqw.azure-api.net
👉🏻 Workspace ID:  81be87b7-cf85-4a4f-9f98-32329b4656d7
👉🏻 App ID:  26d94c4e-7431-41f5-b382-fbb07ebb6f3b


<a id='5'></a>
### 5️⃣ アクセストークンを取得するためのデバイスフローの作成

細かい認可のための注意点:
- APIMの[JWT検証ポリシー](https://learn.microsoft.com/en-us/azure/api-management/validate-azure-ad-token-policy)は、特定のクレーム（トークンに存在する必要がある）をチェックし、細かい認可を適用できます。
- グループクレームは一般的な方法です。このアプローチを使用して認可を駆動できます。ただし、ユーザーがあまりにも多くのグループに属している場合、トークンサイズの制限により`groups`がトークンから除外されます。
- 代替案として、アプリロール定義を構成し、ユーザー/グループにアプリロールを割り当てることができます。このゼロトラスト開発者のベストプラクティスは、柔軟性と制御を向上させ、最小特権でアプリケーションのセキュリティを向上させます。[詳細はこちら](https://learn.microsoft.com/en-us/security/zero-trust/develop/configure-tokens-group-claims-app-roles)。
- `roles`クレームを取得するには、アプリ登録の「APIの公開」セクションに移動します。アプリケーションID URIとスコープを追加します。次に、完全なスコープ（app://<id>/scope）をコピーして、以下のスコープ配列に追加します。
- 「アプリロール」ブレードに移動し、ユーザー/グループメンバー用のアプリロール（例: OpenAI.ChatCompletion）を作成します。次に、テストユーザーまたはグループをアプリロールに割り当てます。
- ログイン後、https://jwt.io/ を使用して`access_token`変数をデコードし、`roles`が送信されていることを確認します。
- 上記の構成を使用して、APIMポリシーに以下のフラグメントを追加し、ユーザーが特定のアプリロールに属していることを確認できます:
```
            <required-claims>
                <claim name="roles" match="any">
                    <value>OpenAI.ChatCompletion</value>
                </claim>
            </required-claims>
```



In [7]:
import json
import logging

import requests
import msal

app = msal.PublicClientApplication(
    client_id, authority="https://login.microsoftonline.com/" + tenant_id)

flow = app.initiate_device_flow(scopes=["User.Read"])
if "user_code" not in flow:
    raise ValueError(
        "Fail to create device flow. Err: %s" % json.dumps(flow, indent=4))

print(flow["message"])



To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BHJZ6EUZU to authenticate.


<a id='6'></a>
### 6️⃣ トークンを取得して Graph API をクエリ

In [12]:
result = app.acquire_token_by_device_flow(flow)

if "access_token" in result:
    access_token = result['access_token']
    print("Access token: %s" % access_token)
    # Calling graph using the access token
    graph_data = requests.get(  # Use token to call downstream service
        "https://graph.microsoft.com/v1.0/me",
        headers={'Authorization': 'Bearer ' + access_token},).json()
    print("Graph API call result: %s" % json.dumps(graph_data, indent=2))
    # print(access_token) # Use a tool like https://jwt.io/ to decode the access token and see its contents
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))  # You may need this when reporting a bug

invalid_grant
AADSTS54005: OAuth2 Authorization code was already redeemed, please retry with a new valid code or use an existing refresh token. Trace ID: d4d36483-e3f7-4fb9-a425-8660ec476100 Correlation ID: 037ad9de-c430-4e39-8491-81a9029ac3cd Timestamp: 2024-10-28 02:41:27Z
037ad9de-c430-4e39-8491-81a9029ac3cd


<a id='requests'></a>
### 🧪 直接HTTPコールを使用したAPIのテスト
Requestsは、ここで生のAPIリクエストを行い、レスポンスを検査するために使用される、エレガントでシンプルなPython用のHTTPライブラリです。

In [9]:
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

messages={"messages":[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}
response = requests.post(url, headers = {'api-key':apim_subscription_key, 'Authorization': 'Bearer ' + access_token}, json = messages)
print("status code: ", response.status_code)
if (response.status_code == 200):
    data = json.loads(response.text)
    print("response: ", data.get("choices")[0].get("message").get("content"))
else:
    print(response.text)


status code:  200
response:  Oh, sorry, I am currently busy not caring about what time it is. But I'm sure you can figure it out on your own, it's not rocket science.


<a id='sdk'></a>
### 🧪 Azure OpenAI Python SDKを使用したAPIのテスト
OpenAPIは広く使用されている[Pythonライブラリ](https://github.com/openai/openai-python)を提供しています。このライブラリには、すべてのリクエストパラメータとレスポンスフィールドの型定義が含まれています。このテストの目的は、APIMがOpenAIへのリクエストをシームレスにプロキシし、その機能を妨げることなく動作することを確認することです。
- 注意: このステップを実行する前に、ターミナルで```pip install openai```を実行してください。

In [11]:

from openai import AzureOpenAI
messages=[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]
client = AzureOpenAI(
    azure_endpoint=apim_resource_gateway_url,
    api_key=apim_subscription_key,
    api_version=openai_api_version        
)
print(access_token)
response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"Authorization": "Bearer " + access_token})
print(response.choices[0].message.content)


eyJ0eXAiOiJKV1QiLCJub25jZSI6ImNLX3psLVlSWkIxTlRfcW9tRFdlcWZxcnlTR2xpQTVmVXc4d1hmVjNnNGMiLCJhbGciOiJSUzI1NiIsIng1dCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCIsImtpZCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCJ9.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTAwMDAtYzAwMC0wMDAwMDAwMDAwMDAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC9mZGI1YmRmZS05MmVkLTRkNWEtYjY5Ny1hNjY0Mzk0NTFiZGEvIiwiaWF0IjoxNzMwMDgyNzY2LCJuYmYiOjE3MzAwODI3NjYsImV4cCI6MTczMDA4Nzk5MiwiYWNjdCI6MCwiYWNyIjoiMSIsImFjcnMiOlsidXJuOnVzZXI6cmVnaXN0ZXJzZWN1cml0eWluZm8iXSwiYWlvIjoiQVZRQXEvOFlBQUFBNThPQ0Y1bU92bVZEUDNpNmxJaTdGUHVXS0g5cndpb3dQL2lvZXdPcXFiOUJTem8xWVp0dTl5anJwRnRGYm9RRzFvTTlOWjNSMXQ0cXZ1aGZ1bkR0OUNaRXBwWk0xV05sL0FMU0FoUGNzYkE9IiwiYW1yIjpbInB3ZCIsIm1mYSJdLCJhcHBfZGlzcGxheW5hbWUiOiJhaS1nYXRld2F5LW9wZW5haS1hcHAiLCJhcHBpZCI6IjFlMzJhNTk2LWUzOWYtNDczOC1hYmMxLTQ5NGMyMjc5NmFlZiIsImFwcGlkYWNyIjoiMCIsImZhbWlseV9uYW1lIjoiQWRtaW5pc3RyYXRvciIsImdpdmVuX25hbWUiOiJTeXN0ZW0iLCJpZHR5cCI6InVzZXIiLCJpcGFkZHIiOiIyNDA0OmY4MDE6ODA1MDozOjM5MTM6OGExYzpkMmU2OjkzMWEiLCJ

<a id='clean'></a>
### 🗑️ リソースのクリーンアップ

ラボが終了したら、追加の料金を避け、Azureサブスクリプションを整理するために、デプロイしたすべてのリソースをAzureから削除する必要があります。
そのためには、[リソースのクリーンアップノートブック](clean-up-resources.ipynb)を使用してください。